# Load DataFrame and Buildup JS-useable table

In [1]:
import os
import pandas as pd
from tqdm.auto import tqdm

In [2]:
local_dir = 'D:\\Sync\\GeoData'

def get(e, key='adcode'):
    if isinstance(e, dict):
        return e.get(key, 0)
    return 0

main_frame = pd.read_json(os.path.join(local_dir, 'main.json'))
main_frame.parent = main_frame.parent.map(get)
main_frame

,geometry,adcode,name,center,centroid,childrenNum,level,parent,subFeatureIndex,acroutes,adchar
0,--,110000,北京市,"[116.405285, 39.904989]","[116.41995, 40.18994]",16.0,province,100000,0.0,[100000],None
1,--,120000,天津市,"[117.190182, 39.125596]","[117.347019, 39.28803]",16.0,province,100000,1.0,[100000],None
2,--,130000,河北省,"[114.502461, 38.045474]",None,11.0,province,100000,2.0,[100000],None
3,--,140000,山西省,"[112.549248, 37.857014]","[112.304761, 37.618555]",11.0,province,100000,3.0,[100000],None
4,--,150000,内蒙古自治区,"[111.670801, 40.818311]","[114.077404, 44.331072]",12.0,province,100000,4.0,[100000],None
...,...,...,...,...,...,...,...,...,...,...,...
3236,--,654322,富蕴县,"[89.524993, 46.993106]","[89.386618, 46.532364]",0.0,district,654300,2.0,"[100000, 650000, 654300]",None
3237,--,654323,福海县,"[87.494569, 47.113128]","[88.046601, 46.362515]",0.0,district,654300,3.0,"[100000, 650000, 654300]",None
3238,--,654324,哈巴河县,"[86.418964, 48.059284]","[86.402485, 48.310203]",0.0,district,654300,4.0,"[100000, 650000, 654300]",None
3239,--,654325,青河县,"[90.381561, 46.672446]","[90.39768, 46.263028]",0.0,district,654300,5.0,"[100000, 650000, 654300]",None


In [22]:
def query(expr, df=main_frame, to_series=True):
    found = df.query(expr)
    print(f'Got {len(found)} records')
    
    if to_series:
        if len(found) == 0:
            print(f'Got ZERO records of "{expr}", but series is required, using None for return')
            return None
        if len(found) > 1:
            print(f'Got MUTIPLE({len(found)}) records of "{expr}", but series is required, using the first one for return')
            return found.iloc[0]
        if len(found) == 1:
            return found.iloc[0]
        
    return found

def trace(i):
    # Get the trace of the [i],
    # [i] refers the index in the [main_frame]
    se = main_frame.iloc[i]
    adcode = se['adcode']
    acroutes = se['acroutes']
    
    df = pd.DataFrame()
    for j in acroutes:
        df = df.append(query(f'adcode == {j}'))
    
    df = df.append(se)
    
    df = pd.concat([df, query(f'parent == {adcode}', to_series=False)])
    
    return df

In [23]:
trace(473)

Got 1 records
Got 1 records
Got 7 records


,acroutes,adchar,adcode,center,centroid,childrenNum,geometry,level,name,parent,subFeatureIndex
34,None,JD,100000.0,None,None,NaN,--,None,,0.0,NaN
30,[100000],None,650000.0,"[87.617733, 43.792818]","[85.294711, 41.371801]",24.0,--,province,新疆维吾尔自治区,100000.0,30.0
473,"[100000, 650000]",None,654300.0,"[88.13963, 47.848393]","[87.048189, 48.013014]",7.0,--,city,阿勒泰地区,650000.0,13.0
3234,"[100000, 650000, 654300]",None,654301.0,"[88.138743, 47.848911]","[87.917843, 47.884663]",0.0,--,district,阿勒泰市,654300.0,0.0
3235,"[100000, 650000, 654300]",None,654321.0,"[86.86186000000001, 47.70453]","[87.230803, 48.313428]",0.0,--,district,布尔津县,654300.0,1.0
3236,"[100000, 650000, 654300]",None,654322.0,"[89.524993, 46.993106]","[89.386618, 46.532364]",0.0,--,district,富蕴县,654300.0,2.0
3237,"[100000, 650000, 654300]",None,654323.0,"[87.494569, 47.113128]","[88.046601, 46.362515]",0.0,--,district,福海县,654300.0,3.0
3238,"[100000, 650000, 654300]",None,654324.0,"[86.418964, 48.059284]","[86.402485, 48.310203]",0.0,--,district,哈巴河县,654300.0,4.0
3239,"[100000, 650000, 654300]",None,654325.0,"[90.381561, 46.672446]","[90.39768, 46.263028]",0.0,--,district,青河县,654300.0,5.0
3240,"[100000, 650000, 654300]",None,654326.0,"[85.876064, 47.434633]","[86.200562, 47.399536]",0.0,--,district,吉木乃县,654300.0,6.0
